In [ ]:
import pandas as pd
import flair.datasets as datasets
from tqdm import tqdm
from src.utils import getEntityInfo
from data.utils import getCandidates

In [ ]:
aida = datasets.NEL_ENGLISH_AIDA()
DBPedia_desc = pd.read_csv('./data/DBPedia.csv')
DBPedia_entites = DBPedia_desc['title'].unique()

In [ ]:
def generate_data(typ):
	set_ = {
		'train': [1, 947, aida.train],
		'dev': [947, 1163, aida.dev],
		'test': [1163, 1394, aida.test]
	}
	entities = []
	for i in range(set_[typ][0], set_[typ][1]):
		cands = getCandidates(i)
		entities += list(cands['url'].unique())
	for i in set_[typ][2]:
		entities += [j.tag for j in i.get_spans()]
	entities = [i.split('/')[-1] for i in set(entities)]
	entities = list(set(entities))
	print(f"Found {len(entities)} unique entities in the {typ} set")
	entity_desc_df = add_desc(entities)
	entity_desc_df.to_csv(f'./data/{typ}.csv', index=False)


def get_desc(entity):
    try:
        return DBPedia_desc[DBPedia_desc['title'] == entity]['description'].values[0]
    except Exception as e:
        return ''


def add_desc(entities):
	missing = []
	entity_desc = []
	print('Adding description from DBPedia...')
	for i in tqdm(entities):
		desc = get_desc(i)
		if desc == '':
			missing.append(i)
		else:
			entity_desc.append([i, desc])
	print('Adding data for missing entities...')
	for i in tqdm(entities):
		info, desc = getEntityInfo(i, inc_extract=True)
		if info != 'Topics referred to by the same term' and desc != '':
			entity_desc.append([i, desc.replace('\n', ' ')])
	return pd.DataFrame(entity_desc, columns=['entity', 'description'])

In [ ]:
generate_data('test')

In [ ]:
generate_data('train')

In [ ]:
generate_data('dev')